In [13]:
cd /Users/Marco/Documents/CSC 478/data/newsgroups5

/Users/Marco/Documents/CSC 478/data/newsgroups5


In [14]:
import kMeans

### 1.a 
** Cosine Similarity Function**

In [51]:
from math import*
 
def square_rooted(x):
    return sqrt(sum([a*a for a in x]))
 
def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return 1-numerator/float(denominator)

### 1.b 
**Loading and Preprocessing** 

In [20]:
import numpy as np
from sklearn.cross_validation import train_test_split

In [78]:
TD = np.loadtxt(open('matrix.txt'),delimiter=",")

In [79]:
TD.shape

(9328, 2500)

Finding the indexes of all the rows that have low DF values 

In [80]:
NRows= TD.shape[0]
list = []
for i in range(NRows-1):
    if (np.count_nonzero(TD[i]))<10:
       list.append(i)

Removing the rows with the respective indexes mentioned above

In [81]:
TD=np.delete(TD,list,axis=0)

In [82]:
TD.shape

(3368, 2500)

**Transposing the data into a doc-term matrix**

In [87]:
dt = TD.T

In [132]:
indices = np.arange(dt.shape[0])

**Splitting the training(80%) and test(20%) data**

In [133]:
train,test,idx1,idx2 = train_test_split(dt,indices, test_size = 0.2, random_state = 33) 

### 1. c
**TFxIDF**

In [134]:
np.set_printoptions(precision=3,edgeitems=4,suppress=True)

In [135]:
numTerms = len(train[0])
NDocs = len(train[:,0])

In [136]:
DF = np.array([(train.T!=0).sum(1)]).T

In [137]:
DF = DF +1

In [138]:
NDocs_train = len(train[:,0])

In [139]:
NMatrix_train = np.ones(np.shape(train.T),dtype=float)*NDocs_train

In [140]:
IDF = np.log2(np.divide(NMatrix_train,DF))

Now that I have the IDF I can use it to obtain the TFxIDF for the train and test matrix

In [142]:
tfidf_train= (train.T*IDF).T

In [143]:
tfidf_test = (test*IDF[:,0])

### 1.d
**kMeans Clustering**

In [144]:
reload(kMeans)

<module 'kMeans' from 'kMeans.pyc'>

In [145]:
centroids, clusters = kMeans.kMeans(tfidf_train, 5, kMeans.cos_dist, kMeans.randCent)

In [225]:
#Removing the error column since I won't be needing it for this problem
clusters=np.delete(clusters,1,1)

In [227]:
#Adding in the doc index as a column to use as a means of IDing
x = np.array([np.concatenate((v,[i])) for i,v in enumerate(clusters)])

The following chuncks of code are my attempt to group the docs into tangible clusters of docs eg. separate the docs into lists depending on what cluster they're in. In order to do this I convert the numpy array into a Pandas Data Frame.

In [230]:
import pandas as pd

In [231]:
labels = ["clusterID","docID"]

In [232]:
df = pd.DataFrame(data=x,columns=labels)

In [234]:
grouped = df.groupby('clusterID')

In [239]:
c0=grouped.get_group(0)
c1=grouped.get_group(1)
c2=grouped.get_group(2)
c3=grouped.get_group(3)
c4=grouped.get_group(4)

In [242]:
#I don't need the clusterID anymore as it's redudant
c0.drop('clusterID', axis=1, inplace=True)
c1.drop('clusterID', axis=1, inplace=True)
c2.drop('clusterID', axis=1, inplace=True)
c3.drop('clusterID', axis=1, inplace=True)
c4.drop('clusterID', axis=1, inplace=True)

/Users/Marco/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Marco/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Marco/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/Marco/anaconda/lib/python2.7

Once I'm done grouping, I convert the "clusters" back into (1-D) numpy arrays...

In [243]:
c0=(c0.values).flatten()
c1=(c1.values).flatten()
c2=(c2.values).flatten()
c3=(c3.values).flatten()
c4=(c4.values).flatten()

In [268]:
#Convert values to int as I will be using them to index 
c0=c0.astype(int)
c1=c1.astype(int)
c2=c2.astype(int)
c3=c3.astype(int)
c4=c4.astype(int)

In [269]:
docCl=[]
docCl.append(c0)
docCl.append(c1)
docCl.append(c2)
docCl.append(c3)
docCl.append(c4)

I now have an organized version of the original clusters array. docCl is a collection (narrays) of clusters (narray) that each have their respective documents (indexes) in them. 

In [270]:
docCl=np.array(docCl)

In [263]:
# The words from the text file
terms=np.genfromtxt('terms.txt',dtype=str)

In [266]:
import heapq

In [281]:
def display_info(clusters,centroids,docTerm,terms,N=10):
    
    #Collecting index of topN terms in centroid
    #for each centroid.
    terms_in_centroid=[] 
    for c in centroids:
        index_of_terms=heapq.nlargest(N,range(len(c)),c.take)
        terms_in_centroid.append(index_of_terms)
    
    #Computing the DF for each cluster 
    DFList=[]
    for c in clusters:
        temp=docTerm[[c]]
        DF = np.array([(temp.T!=0).sum(1)])
        DFList.append(DF.flatten())
    
    #Display
    for i,c in enumerate(terms_in_centroid):
        sizeOfCluster = len(clusters[i])
        print "Cluster: %d Size: %d"%(i,sizeOfCluster)
        for term in c:
            DF = DFList[i][int(term)]
            word = terms[int(term)]
            percent = int((DF/float(sizeOfCluster))*100)
            print "Term: %s\t Cluster DF:%d\t Percent: %d"%(word,DF,percent)
        print
    

In [282]:
display_info(docCl,centroids,train,terms,5)

Cluster: 0 Size: 9
Term: altath	 Cluster DF:8	 Percent: 88
Term: csugradcsvtedu	 Cluster DF:7	 Percent: 77
Term: diskman	 Cluster DF:7	 Percent: 77
Term: glacial	 Cluster DF:6	 Percent: 66
Term: dsuvaxdsuedu	 Cluster DF:6	 Percent: 66

Cluster: 1 Size: 165
Term: chevi	 Cluster DF:97	 Percent: 58
Term: closest	 Cluster DF:36	 Percent: 21
Term: exit	 Cluster DF:64	 Percent: 38
Term: connector	 Cluster DF:66	 Percent: 40
Term: dinari	 Cluster DF:52	 Percent: 31

Cluster: 2 Size: 804
Term: gizbj	 Cluster DF:297	 Percent: 36
Term: carri	 Cluster DF:160	 Percent: 19
Term: bottom	 Cluster DF:159	 Percent: 19
Term: engumdedu	 Cluster DF:229	 Percent: 28
Term: branch	 Cluster DF:83	 Percent: 10

Cluster: 3 Size: 233
Term: chevi	 Cluster DF:117	 Percent: 50
Term: australian	 Cluster DF:105	 Percent: 45
Term: author	 Cluster DF:64	 Percent: 27
Term: cach	 Cluster DF:48	 Percent: 20
Term: clean	 Cluster DF:29	 Percent: 12

Cluster: 4 Size: 789
Term: biggest	 Cluster DF:3	 Percent: 0
Term: content	

### 1e. 
**Completeness and Homogeneity** 

In [149]:
predict = clusters[:,0]

In [124]:
actual = np.loadtxt(open('classes.txt'),skiprows=1)

In [127]:
actual = actual[:,1]

In [150]:
predict = predict.astype(int)
actual  = actual.astype(int)

In [166]:
actual = [actual[i] for i in idx1]

In [130]:
from sklearn.metrics import completeness_score, homogeneity_score

In [171]:
print completeness_score(actual,predict)

0.719354899289


In [172]:
print homogeneity_score(actual,predict)

0.542625793855


### 1.f
**Testing the training model**

In [174]:
from scipy import spatial

In [210]:
test_clusters=[]
for d in tfidf_test:
    dist = []
    for c in centroids:
        dist.append(spatial.distance.cosine(d,c))
    doc = (dist.index(min(dist)),dist)
    test_clusters.append(doc)

In [221]:
file = open("part2f.txt", "w")
for doc in test_clusters:
    file.write("%s%s\n" % doc)
file.close()